# Learning Rate: Local vs Global Minimum

Interactive visualization of error landscape with local and global minima

In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Create error surface with multiple local minima and one global minimum
def error_surface(x, y):
    """Error function with multiple local minima"""
    # Main bowl (global minimum)
    term1 = 0.5 * ((x - 0.5)**2 + (y - 0.5)**2)
    
    # Add local minima using sine waves
    term2 = 10 * np.sin(2 * np.pi * x) * np.sin(2 * np.pi * y)
    
    # Another local minimum
    term3 = 15 * np.exp(-((x + 2)**2 + (y + 2)**2))
    
    return term1 + term2 + term3 + 20

In [3]:
# Generate grid
x = np.linspace(-4, 4, 100)
y = np.linspace(-4, 4, 100)
X, Y = np.meshgrid(x, y)
Z = error_surface(X, Y)

# Define minima locations
global_min = {'x': 0.5, 'y': 0.5, 'z': error_surface(0.5, 0.5), 'label': 'Global Minimum'}
local_mins = [
    {'x': -2.0, 'y': -2.0, 'z': error_surface(-2.0, -2.0), 'label': 'Local Minimum 1'},
    {'x': 1.25, 'y': -0.75, 'z': error_surface(1.25, -0.75), 'label': 'Local Minimum 2'},
]

In [4]:
# Create interactive 3D surface plot
fig = go.Figure()

# Add surface
fig.add_trace(go.Surface(
    x=X, y=Y, z=Z,
    colorscale='Viridis',
    name='Error Surface',
    showscale=True,
    opacity=0.9
))

# Add global minimum marker
fig.add_trace(go.Scatter3d(
    x=[global_min['x']],
    y=[global_min['y']],
    z=[global_min['z']],
    mode='markers+text',
    marker=dict(size=15, color='red', symbol='diamond'),
    text=[global_min['label']],
    textposition='top center',
    name='Global Minimum',
    textfont=dict(size=12, color='red')
))

# Add local minima markers
for i, lm in enumerate(local_mins):
    fig.add_trace(go.Scatter3d(
        x=[lm['x']],
        y=[lm['y']],
        z=[lm['z']],
        mode='markers+text',
        marker=dict(size=12, color='yellow', symbol='circle'),
        text=[lm['label']],
        textposition='top center',
        name=lm['label'],
        textfont=dict(size=10, color='yellow')
    ))

fig.update_layout(
    title='Error Surface: Local vs Global Minima',
    scene=dict(
        xaxis_title='Parameter 1',
        yaxis_title='Parameter 2',
        zaxis_title='Training Error',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.3)
        )
    ),
    width=900,
    height=700,
    showlegend=True
)

fig.show()

## Gradient Descent Simulation with Different Learning Rates

In [5]:
def gradient(x, y, h=1e-5):
    """Compute gradient using finite differences"""
    dx = (error_surface(x + h, y) - error_surface(x - h, y)) / (2 * h)
    dy = (error_surface(x, y + h) - error_surface(x, y - h)) / (2 * h)
    return dx, dy

def gradient_descent(start_x, start_y, learning_rate, max_iter=100):
    """Perform gradient descent"""
    path_x = [start_x]
    path_y = [start_y]
    path_z = [error_surface(start_x, start_y)]
    
    x, y = start_x, start_y
    
    for _ in range(max_iter):
        dx, dy = gradient(x, y)
        
        # Update with learning rate
        x = x - learning_rate * dx
        y = y - learning_rate * dy
        
        path_x.append(x)
        path_y.append(y)
        path_z.append(error_surface(x, y))
        
        # Early stopping if gradient is very small
        if np.sqrt(dx**2 + dy**2) < 1e-4:
            break
    
    return path_x, path_y, path_z

In [6]:
# Interactive widget for learning rate
def plot_gradient_descent(learning_rate, start_x, start_y):
    # Run gradient descent
    path_x, path_y, path_z = gradient_descent(start_x, start_y, learning_rate)
    
    # Create figure
    fig = go.Figure()
    
    # Add surface
    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        name='Error Surface',
        showscale=True,
        opacity=0.7
    ))
    
    # Add global minimum
    fig.add_trace(go.Scatter3d(
        x=[global_min['x']],
        y=[global_min['y']],
        z=[global_min['z']],
        mode='markers',
        marker=dict(size=15, color='red', symbol='diamond'),
        name='Global Minimum'
    ))
    
    # Add local minima
    for lm in local_mins:
        fig.add_trace(go.Scatter3d(
            x=[lm['x']],
            y=[lm['y']],
            z=[lm['z']],
            mode='markers',
            marker=dict(size=12, color='yellow', symbol='circle'),
            name='Local Minimum'
        ))
    
    # Add gradient descent path
    fig.add_trace(go.Scatter3d(
        x=path_x,
        y=path_y,
        z=path_z,
        mode='lines+markers',
        marker=dict(size=4, color='white'),
        line=dict(color='white', width=4),
        name=f'GD Path (LR={learning_rate})'
    ))
    
    # Add start point
    fig.add_trace(go.Scatter3d(
        x=[path_x[0]],
        y=[path_y[0]],
        z=[path_z[0]],
        mode='markers',
        marker=dict(size=10, color='cyan', symbol='cross'),
        name='Start Point'
    ))
    
    # Add end point
    fig.add_trace(go.Scatter3d(
        x=[path_x[-1]],
        y=[path_y[-1]],
        z=[path_z[-1]],
        mode='markers',
        marker=dict(size=10, color='magenta', symbol='x'),
        name='End Point'
    ))
    
    # Determine outcome
    final_error = path_z[-1]
    if final_error < global_min['z'] + 2:
        outcome = "✓ Reached Global Minimum"
        outcome_color = "green"
    else:
        outcome = "✗ Stuck in Local Minimum"
        outcome_color = "red"
    
    fig.update_layout(
        title=f'Learning Rate = {learning_rate} | Steps: {len(path_x)-1} | {outcome}',
        scene=dict(
            xaxis_title='Parameter 1',
            yaxis_title='Parameter 2',
            zaxis_title='Training Error',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3))
        ),
        width=900,
        height=700
    )
    
    fig.show()
    
    print(f"\nFinal Error: {final_error:.2f}")
    print(f"Final Position: ({path_x[-1]:.2f}, {path_y[-1]:.2f})")
    
# Create interactive widgets
learning_rate_widget = widgets.FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Learning Rate:',
    style={'description_width': 'initial'}
)

start_x_widget = widgets.FloatSlider(
    value=-3.0,
    min=-4.0,
    max=4.0,
    step=0.1,
    description='Start X:',
    style={'description_width': 'initial'}
)

start_y_widget = widgets.FloatSlider(
    value=3.0,
    min=-4.0,
    max=4.0,
    step=0.1,
    description='Start Y:',
    style={'description_width': 'initial'}
)

# Create interactive plot
interactive_plot = widgets.interactive(
    plot_gradient_descent,
    learning_rate=learning_rate_widget,
    start_x=start_x_widget,
    start_y=start_y_widget
)

display(interactive_plot)

interactive(children=(FloatSlider(value=0.1, description='Learning Rate:', max=0.5, min=0.01, step=0.01, style…

## Comparison: Different Learning Rates from Same Starting Point

In [7]:
# Compare different learning rates
start_x, start_y = -3.0, 3.0
learning_rates = [0.02, 0.1, 0.3]
colors = ['cyan', 'lime', 'magenta']

fig = go.Figure()

# Add surface
fig.add_trace(go.Surface(
    x=X, y=Y, z=Z,
    colorscale='Viridis',
    name='Error Surface',
    showscale=True,
    opacity=0.6
))

# Add global minimum
fig.add_trace(go.Scatter3d(
    x=[global_min['x']],
    y=[global_min['y']],
    z=[global_min['z']],
    mode='markers+text',
    marker=dict(size=15, color='red', symbol='diamond'),
    text=['Global Min'],
    name='Global Minimum'
))

# Add paths for different learning rates
for lr, color in zip(learning_rates, colors):
    path_x, path_y, path_z = gradient_descent(start_x, start_y, lr)
    
    fig.add_trace(go.Scatter3d(
        x=path_x,
        y=path_y,
        z=path_z,
        mode='lines+markers',
        marker=dict(size=3, color=color),
        line=dict(color=color, width=3),
        name=f'LR = {lr}'
    ))

# Add start point
fig.add_trace(go.Scatter3d(
    x=[start_x],
    y=[start_y],
    z=[error_surface(start_x, start_y)],
    mode='markers+text',
    marker=dict(size=12, color='white', symbol='cross'),
    text=['Start'],
    name='Start Point'
))

fig.update_layout(
    title='Comparison of Different Learning Rates',
    scene=dict(
        xaxis_title='Parameter 1',
        yaxis_title='Parameter 2',
        zaxis_title='Training Error',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.3))
    ),
    width=900,
    height=700
)

fig.show()

## Key Insights

**Too Large Learning Rate:**
- May overshoot and oscillate
- Can get stuck bouncing around a local minimum
- Might miss the global minimum entirely

**Too Small Learning Rate:**
- Slow convergence
- Gets stuck in first local minimum encountered
- Cannot escape shallow local minima

**Optimal Learning Rate:**
- Fast enough to escape local minima
- Controlled enough to converge to global minimum
- Balance between exploration and exploitation